In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.utils.data import Dataset,DataLoader

'''
STEP 1: LOADING DATASET
'''
class YelpDataset(Dataset):
    def __init__(self, path, train=True):
        super(YelpDataset, self).__init__()
        self.train = train
        
        datas = pd.read_json(path, lines=True).fillna(0)
        y = datas['label']
        x = datas.drop(columns=['business_id', 'label'])
        x['city'] = preprocessing.LabelEncoder().fit_transform(x['city'])
        x['state'] = preprocessing.LabelEncoder().fit_transform(x['state'])
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)
        if train:
            self.X = X_train
            self.y = y_train
        else:
            self.X = X_test
            self.y = y_test
    
    def __getitem__(self, index):
        return torch.tensor(self.X.iloc[index], dtype=torch.float32), self.y.iloc[index]

    def __len__(self):
        return len(self.X)

train_dataset = YelpDataset('../yelp_dataset/yelp_dataset_labeled.json', train=True)
test_dataset = YelpDataset('../yelp_dataset/yelp_dataset_labeled.json', train=False)

FileNotFoundError: [WinError 206] The filename or extension is too long: 'C:\\Users\\Jack\\miniconda3\\envs\\py38-ds\\lib\\site-packages\\torch\\lib'

In [131]:
'''
STEP 2: MAKING DATASET ITERABLE
'''

batch_size = 100
n_iters = 1050
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [132]:
'''
STEP 3: CREATE MODEL CLASS
'''
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Linear function 1: 363 --> 100
        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        self.drop = nn.Dropout(0.2)
        # Non-linearity 
        self.relu = nn.ReLU()

        # Linear function 2 (readout): 100 --> 2
        self.fc2 = nn.Linear(hidden_dim, output_dim)  

    def forward(self, x):
        # Linear function 1
        out = self.fc1(x)
        out = self.drop(out)
        # Non-linearity 1
        out = self.relu(out)

        # Linear function 2
        out = self.fc2(out)

        return out

In [133]:
'''
STEP 4: INSTANTIATE MODEL CLASS
'''
input_dim = 1*10
hidden_dim = 16
output_dim = 2

model = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)

In [134]:
'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss()


In [135]:
'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [1]:

'''
STEP 7: TRAIN THE MODEL
'''
iter = 0
for epoch in range(num_epochs):
    for i, (X, y) in enumerate(train_loader):
        # Load X with gradient accumulation capabilities
        X = X.view(-1, input_dim).requires_grad_()

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(X)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, y)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 100 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for X, y in test_loader:
                # Load X with gradient accumulation capabilities
                X = X.view(-1, input_dim).requires_grad_()

                # Forward pass only to get logits/output
                outputs = model(X)

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                print(predicted)
                print(type(predicted))
                # Total number of labels
                total += y.size(0)

                # Total correct predictions
                correct += (predicted == y).sum()

            accuracy = 100 * correct / total

            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

NameError: name 'num_epochs' is not defined